In [1]:
import pandas as pd
from src.dimension_modeler import Dimension_Modeler
from src.util import update_start_date


### DIM PRODUCTS
- product_id (int) (Natural Key)
- product_key (int) (Surrogate Key)
- product_name (str)
- product_category (str) DONE
- product_price (decimal 10,2) DONE
- product_cost (decimal 10,2) DONE 
- is_current(boolean) DONE
- start_date (date) DONE 
- end_date (date) DONE

In [2]:
products_df = pd.read_csv('datasets/donut_shop_product_info.csv')

In [3]:
products_df

,Product Category,Product Price,Product Cost,Historical Cost
0,Donut,1,0.10,0.1
1,Hot Foods,3,0.80,0.5
2,Drinks,2,1.15,1.0


#### Getting historical prices and dates associated

In [4]:
transactions_df = pd.read_csv('datasets/donut_shop_transactions.csv')

In [5]:
price_history = []


In [6]:
for x in transactions_df.groupby('Category')['Price per Unit'].value_counts().index:
    price_history.append({
            'category': x[0],
            'price': x[1],
            'start_date': transactions_df[transactions_df['Price per Unit'] == x[1]]['Date'].min(),
            'end_date': transactions_df[transactions_df['Price per Unit'] == x[1]]['Date'].max(),
            }
                         )

In [7]:
price_history

[{'category': 'Donut',
  'price': 1.0,
  'start_date': '2020-01-01 00:27:34',
  'end_date': '2024-12-31 23:53:27'},
 {'category': 'Donut',
  'price': 0.5,
  'start_date': '2015-01-01 01:35:44',
  'end_date': '2019-12-31 23:49:08'},
 {'category': 'Drinks',
  'price': 2.0,
  'start_date': '2020-01-01 03:05:09',
  'end_date': '2024-12-31 23:53:27'},
 {'category': 'Drinks',
  'price': 1.25,
  'start_date': '2015-01-01 00:56:32',
  'end_date': '2019-12-31 23:49:08'},
 {'category': 'Hot Foods',
  'price': 3.0,
  'start_date': '2020-01-01 00:27:34',
  'end_date': '2024-12-31 23:53:27'},
 {'category': 'Hot Foods',
  'price': 1.5,
  'start_date': '2015-01-01 01:41:45',
  'end_date': '2019-12-31 23:32:46'}]

In [8]:
transactions_df[transactions_df['Price per Unit'] == 1.00]['Date'].max()

'2024-12-31 23:53:27'

In [9]:
transactions_df[transactions_df['Price per Unit'] == 1.00]['Date'].min()

'2020-01-01 00:27:34'

In [10]:
dim_prod_df = pd.DataFrame(price_history)
    

#### Getting is_current field

In [11]:
dim_prod_df

,category,price,start_date,end_date
0,Donut,1.00,2020-01-01 00:27:34,2024-12-31 23:53:27
1,Donut,0.50,2015-01-01 01:35:44,2019-12-31 23:49:08
2,Drinks,2.00,2020-01-01 03:05:09,2024-12-31 23:53:27
3,Drinks,1.25,2015-01-01 00:56:32,2019-12-31 23:49:08
4,Hot Foods,3.00,2020-01-01 00:27:34,2024-12-31 23:53:27
5,Hot Foods,1.50,2015-01-01 01:41:45,2019-12-31 23:32:46


In [12]:
dim_prod_df['max_start_date'] = dim_prod_df.groupby('category')['start_date'].transform('max')
dim_prod_df

,category,price,start_date,end_date,max_start_date
0,Donut,1.00,2020-01-01 00:27:34,2024-12-31 23:53:27,2020-01-01 00:27:34
1,Donut,0.50,2015-01-01 01:35:44,2019-12-31 23:49:08,2020-01-01 00:27:34
2,Drinks,2.00,2020-01-01 03:05:09,2024-12-31 23:53:27,2020-01-01 03:05:09
3,Drinks,1.25,2015-01-01 00:56:32,2019-12-31 23:49:08,2020-01-01 03:05:09
4,Hot Foods,3.00,2020-01-01 00:27:34,2024-12-31 23:53:27,2020-01-01 00:27:34
5,Hot Foods,1.50,2015-01-01 01:41:45,2019-12-31 23:32:46,2020-01-01 00:27:34


In [13]:
dim_prod_df['is_current'] = dim_prod_df['max_start_date'] == dim_prod_df['start_date']
dim_prod_df

,category,price,start_date,end_date,max_start_date,is_current
0,Donut,1.00,2020-01-01 00:27:34,2024-12-31 23:53:27,2020-01-01 00:27:34,True
1,Donut,0.50,2015-01-01 01:35:44,2019-12-31 23:49:08,2020-01-01 00:27:34,False
2,Drinks,2.00,2020-01-01 03:05:09,2024-12-31 23:53:27,2020-01-01 03:05:09,True
3,Drinks,1.25,2015-01-01 00:56:32,2019-12-31 23:49:08,2020-01-01 03:05:09,False
4,Hot Foods,3.00,2020-01-01 00:27:34,2024-12-31 23:53:27,2020-01-01 00:27:34,True
5,Hot Foods,1.50,2015-01-01 01:41:45,2019-12-31 23:32:46,2020-01-01 00:27:34,False


In [14]:
dim_prod_df.drop(columns='max_start_date', inplace=True)


#### Updating end_date

In [15]:
dim_prod_df

,category,price,start_date,end_date,is_current
0,Donut,1.00,2020-01-01 00:27:34,2024-12-31 23:53:27,True
1,Donut,0.50,2015-01-01 01:35:44,2019-12-31 23:49:08,False
2,Drinks,2.00,2020-01-01 03:05:09,2024-12-31 23:53:27,True
3,Drinks,1.25,2015-01-01 00:56:32,2019-12-31 23:49:08,False
4,Hot Foods,3.00,2020-01-01 00:27:34,2024-12-31 23:53:27,True
5,Hot Foods,1.50,2015-01-01 01:41:45,2019-12-31 23:32:46,False


In [16]:
max_date = pd.Timestamp.max.strftime('%Y-%m-%d %H:%M:%S')
max_date

'2262-04-11 23:47:16'

In [17]:
dim_prod_df.loc[dim_prod_df['is_current'], 'end_date'] = max_date

#### Updating Start_date

In [18]:
dim_prod_df

,category,price,start_date,end_date,is_current
0,Donut,1.00,2020-01-01 00:27:34,2262-04-11 23:47:16,True
1,Donut,0.50,2015-01-01 01:35:44,2019-12-31 23:49:08,False
2,Drinks,2.00,2020-01-01 03:05:09,2262-04-11 23:47:16,True
3,Drinks,1.25,2015-01-01 00:56:32,2019-12-31 23:49:08,False
4,Hot Foods,3.00,2020-01-01 00:27:34,2262-04-11 23:47:16,True
5,Hot Foods,1.50,2015-01-01 01:41:45,2019-12-31 23:32:46,False


In [19]:
dim_prod_df = dim_prod_df.groupby('category', group_keys=False).apply(update_start_date)

In [20]:
dim_prod_df

,category,price,start_date,end_date,is_current
0,Donut,1.00,2019-12-31 23:49:08,2262-04-11 23:47:16,True
1,Donut,0.50,2015-01-01 01:35:44,2019-12-31 23:49:08,False
2,Drinks,2.00,2019-12-31 23:49:08,2262-04-11 23:47:16,True
3,Drinks,1.25,2015-01-01 00:56:32,2019-12-31 23:49:08,False
4,Hot Foods,3.00,2019-12-31 23:32:46,2262-04-11 23:47:16,True
5,Hot Foods,1.50,2015-01-01 01:41:45,2019-12-31 23:32:46,False


#### Adding Historical prices


In [21]:
dim_prod_df

,category,price,start_date,end_date,is_current
0,Donut,1.00,2019-12-31 23:49:08,2262-04-11 23:47:16,True
1,Donut,0.50,2015-01-01 01:35:44,2019-12-31 23:49:08,False
2,Drinks,2.00,2019-12-31 23:49:08,2262-04-11 23:47:16,True
3,Drinks,1.25,2015-01-01 00:56:32,2019-12-31 23:49:08,False
4,Hot Foods,3.00,2019-12-31 23:32:46,2262-04-11 23:47:16,True
5,Hot Foods,1.50,2015-01-01 01:41:45,2019-12-31 23:32:46,False


In [22]:
products_df

,Product Category,Product Price,Product Cost,Historical Cost
0,Donut,1,0.10,0.1
1,Hot Foods,3,0.80,0.5
2,Drinks,2,1.15,1.0


In [23]:
dim_prod_df = pd.merge(dim_prod_df, products_df, how='left', left_on='category', right_on='Product Category')

In [24]:
dim_prod_df = dim_prod_df[['category', 'price', 'start_date', 'end_date', 'is_current', 'Product Cost', 'Historical Cost']]

In [25]:
dim_prod_df

,category,price,start_date,end_date,is_current,Product Cost,Historical Cost
0,Donut,1.00,2019-12-31 23:49:08,2262-04-11 23:47:16,True,0.10,0.1
1,Donut,0.50,2015-01-01 01:35:44,2019-12-31 23:49:08,False,0.10,0.1
2,Drinks,2.00,2019-12-31 23:49:08,2262-04-11 23:47:16,True,1.15,1.0
3,Drinks,1.25,2015-01-01 00:56:32,2019-12-31 23:49:08,False,1.15,1.0
4,Hot Foods,3.00,2019-12-31 23:32:46,2262-04-11 23:47:16,True,0.80,0.5
5,Hot Foods,1.50,2015-01-01 01:41:45,2019-12-31 23:32:46,False,0.80,0.5


In [26]:
dim_prod_df.loc[~dim_prod_df['is_current'], 'Product Cost'] = dim_prod_df['Historical Cost']
dim_prod_df.drop(columns='Historical Cost', inplace=True)

In [27]:
dim_prod_df

,category,price,start_date,end_date,is_current,Product Cost
0,Donut,1.00,2019-12-31 23:49:08,2262-04-11 23:47:16,True,0.10
1,Donut,0.50,2015-01-01 01:35:44,2019-12-31 23:49:08,False,0.10
2,Drinks,2.00,2019-12-31 23:49:08,2262-04-11 23:47:16,True,1.15
3,Drinks,1.25,2015-01-01 00:56:32,2019-12-31 23:49:08,False,1.00
4,Hot Foods,3.00,2019-12-31 23:32:46,2262-04-11 23:47:16,True,0.80
5,Hot Foods,1.50,2015-01-01 01:41:45,2019-12-31 23:32:46,False,0.50


In [28]:
products_df

,Product Category,Product Price,Product Cost,Historical Cost
0,Donut,1,0.10,0.1
1,Hot Foods,3,0.80,0.5
2,Drinks,2,1.15,1.0


#### Adding product name

In [29]:
dim_prod_df

,category,price,start_date,end_date,is_current,Product Cost
0,Donut,1.00,2019-12-31 23:49:08,2262-04-11 23:47:16,True,0.10
1,Donut,0.50,2015-01-01 01:35:44,2019-12-31 23:49:08,False,0.10
2,Drinks,2.00,2019-12-31 23:49:08,2262-04-11 23:47:16,True,1.15
3,Drinks,1.25,2015-01-01 00:56:32,2019-12-31 23:49:08,False,1.00
4,Hot Foods,3.00,2019-12-31 23:32:46,2262-04-11 23:47:16,True,0.80
5,Hot Foods,1.50,2015-01-01 01:41:45,2019-12-31 23:32:46,False,0.50


In [30]:
products_list_df = transactions_df[['Product Name', 'Category']].drop_duplicates(keep='first').reset_index().drop(columns='index')

In [31]:
products_list_df

,Product Name,Category
0,Coke,Drinks
1,Blueberry,Donut
2,Cinnamon Swirl,Donut
3,Croissant Hot Dog,Hot Foods
4,Biscuits,Hot Foods
5,Sprite,Drinks
6,Blueberry Donut Hole,Donut
7,Glaze,Donut
8,Chocolate,Donut
9,Tropicana,Drinks


In [ ]:
dim_prod_df = pd.merge(dim_prod_df, products_list_df, how='left', left_on='category', right_on='Category')


In [35]:
dim_prod_df.sample(10)


,category,price,start_date,end_date,is_current,Product Cost,Product Name,Category
40,Drinks,1.25,2015-01-01 00:56:32,2019-12-31 23:49:08,False,1.00,Coffee,Drinks
47,Hot Foods,3.00,2019-12-31 23:32:46,2262-04-11 23:47:16,True,0.80,Ham and Cheese,Hot Foods
35,Drinks,2.00,2019-12-31 23:49:08,2262-04-11 23:47:16,True,1.15,Diet Coke,Drinks
0,Donut,1.00,2019-12-31 23:49:08,2262-04-11 23:47:16,True,0.10,Blueberry,Donut
23,Donut,0.50,2015-01-01 01:35:44,2019-12-31 23:49:08,False,0.10,Boston Creme,Donut
31,Drinks,2.00,2019-12-31 23:49:08,2262-04-11 23:47:16,True,1.15,Starbucks Caramel,Drinks
1,Donut,1.00,2019-12-31 23:49:08,2262-04-11 23:47:16,True,0.10,Cinnamon Swirl,Donut
50,Hot Foods,1.50,2015-01-01 01:41:45,2019-12-31 23:32:46,False,0.50,Biscuits,Hot Foods
42,Drinks,1.25,2015-01-01 00:56:32,2019-12-31 23:49:08,False,1.00,Starbucks Mocha,Drinks
27,Donut,0.50,2015-01-01 01:35:44,2019-12-31 23:49:08,False,0.10,Daring Donut,Donut


### DIM EMPLOYEES
- employee_id (int) DONE 
- employee_name (str) DONE 
- job_title (str) DONE 

In [2]:
employee_df = pd.read_csv('datasets/donut_shop_employees.csv')


In [3]:
dim_model = Dimension_Modeler(employee_df)

In [4]:
employee_df.columns = ['employee_name', 'job_title']

In [5]:
employee_df.dtypes

employee_name    object
job_title        object
dtype: object

In [6]:
employee_df = dim_model.make_id_col('employee_id')

In [7]:
employee_df.dtypes

employee_id       int64
employee_name    object
job_title        object
dtype: object

In [8]:
employee_df

,employee_id,employee_name,job_title
0,1,Alice,Owner
1,2,Bob,Owner
2,3,Charlie,Owner
3,4,David,Cashier
4,5,Eva,Cashier
5,6,Frank,Cashier
6,7,Grace,Cashier
7,8,Hannah,Cashier


In [9]:
dim_model.make_csv('dim_employees')

dim_employees.csv created in datasets/dimensions/
